<a href="https://colab.research.google.com/github/nlpbokproject/NLP_BOK_Team/blob/master/EdailyScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

     |████████████████████████████████| 911kB 2.9MB/s 


# 이데일리 기사 크롤링

# 모듈 import

In [0]:
from bs4 import BeautifulSoup, Comment
from urllib.request import urlopen
import pandas as pd
from selenium import webdriver
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests

# 크롤링을 원하는 날자 범위 지정

In [0]:
import datetime, time

#start_date = datetime.datetime(2017,1,1)
#end_date = datetime.datetime(2017,12,31)

start_date = datetime.datetime.strptime("01-03-2017", "%d-%m-%Y") #시작 날짜
end_date = datetime.datetime.strptime("31-03-2017", "%d-%m-%Y") #종료 날짜

date_generated = [start_date + datetime.timedelta(days=x) for x in range(0, (end_date-start_date).days+1)]
date_list = []
for date in date_generated:
    #print(date.strftime("%d-%m-%Y"))
    nowdate = str(date.year)+"."+str(date.month).zfill(2)+"."+str(date.day).zfill(2)
    print(nowdate)
    date_list.append("https://search.naver.com/search.naver?where=news&query=%EA%B8%88%EB%A6%AC&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds="+nowdate+"&de="+nowdate+"&docid=&nso=so%3Ar%2Cp%3A%2Ca%3Aall&mynews=0&refresh_start=0&related=0")


# 원하는 날짜 범위 내의 '금리'가 들어간 기사(이데일리 기사만) 검색

In [0]:
tmp =[]
driver = webdriver.Chrome("chromedriver.exe")
for date in date_list:
    driver.get(date)
    driver.set_page_load_timeout(30)
    driver.find_element_by_xpath("""//*[@id="news_popup"]/a""").click()
    time.sleep(1)
    driver.find_element_by_xpath("""//*[@id="ca_1018"]""").click()
    time.sleep(1)
    driver.find_element_by_xpath("""//*[@id="_nx_option_media"]/div[2]/div[3]/button[1]/span""").click()
    time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    a_tags = soup.find_all('a', class_='_sp_each_title')
    print(a_tags[0].text)
    
    tmp.append(a_tags)
    
    while soup.find('a', class_='next') is not None:
        next_click = soup.find('a', class_='next')
        url = 'https://search.naver.com/search.naver' + str(next_click['href'])
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        a_tags = soup.find_all('a', class_='_sp_each_title')
        tmp.append(a_tags)
        time.sleep(1)
        print(a_tags[0].text)

WebDriverException: ignored

# 각 URL에서의 기사내용 수집(기사제목, URL, 날짜, 본문)

In [0]:
date=''
prepro=''
cnt= 1
for j in range(len(tmp)):
    for i in tmp[j]:
         #제목
            print("제목 : " + i.get_text())
            title = i.get_text()
            #URL
            url = str(i['href'])
            response = requests.get(url)
            soup = BeautifulSoup(response.text,'html.parser')
            #print("URL : " + url)
            #날짜
            if soup.find('div', class_='dates') is not None:
                date = soup.find('div', class_='dates').find('p')
                date = str(date)
                date = date[6:16]
                print(date)
            #print("date : "+ date)
            #본문
            if soup.find('div', class_='news_body') is not None:
                div_tag = soup.find('div', class_='news_body')
                prepro = str(div_tag)
                prepro = re.sub('\n','',prepro,0).strip()
                prepro = re.sub('<!--.+?-->','',prepro,0).strip()
                prepro = re.sub('<.+?>','',prepro,0).strip()
            #print(prepro)
            #print('',end='\n\n')
            new_df = pd.DataFrame({'com':'Edaily','title':title, 'date':date, 'url':url, 'script':prepro}, index=[0])
            df1 = pd.concat([df1, new_df])
    


# 수집 데이터 csv파일에 저장


In [0]:
df1.to_csv("./Edaily/edaily_03.csv", encoding='UTF8')